In [24]:
import os
import pandas as pd
import time
import random
import getpass
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from bs4 import BeautifulSoup
from tqdm import tqdm

# SETTING DRIVER

```python
def driver():
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    s = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--window-size=1920,1080")
    return webdriver.Chrome(service=s, options=options)
```

In [25]:
from selenium import webdriver

# Configurar el WebDriver para Google Chrome
options = webdriver.ChromeOptions()
options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
wd = webdriver.Chrome(options=options)

# Abrir una página web
url ='https://www.linkedin.com/jobs/search/?currentJobId=3652284026&distance=25&geoId=107025191&keywords=data%20analyst&refresh=true'

wd.get(url)

no_of_jobs = int(wd.find_element(By.CSS_SELECTOR, "h1>span").get_attribute('innerText'))

wd.find_element(By.CSS_SELECTOR, '.jobs-search__results-list')

The version of chrome cannot be detected. Trying with latest driver version


<selenium.webdriver.remote.webelement.WebElement (session="0f0c817639baa1603220c339af148161", element="7FF4C1D2CE68F7CC5EAD4B10EB906465_element_19")>

In [26]:
wd.get(url)

# Browse all the jobs

In [27]:
no_of_jobs = int(wd.find_element(By.CSS_SELECTOR, "h1>span").get_attribute('innerText'))

In [28]:
wd.find_element(By.CSS_SELECTOR, '.jobs-search__results-list')

<selenium.webdriver.remote.webelement.WebElement (session="0f0c817639baa1603220c339af148161", element="5DF6B02FF10E9E1F02D20A14D5FB89C3_element_33")>

In [29]:
no_of_jobs

2000

In [30]:
def enconding_url (job_title, location, base_url):
    
    base_url = "https://www.linkedin.com/jobs/search"
    
    params = {
        "keywords": job_title,
        "location": location,
    }

    encoded_params = "&" + "&".join([f"{quote(k)}={quote(v)}" for k, v in params.items()])
    url_with_params = f"{base_url}{encoded_params}"

    return url_with_params

In [31]:
from selenium import webdriver
import os
import pandas as pd
import time
import random
import getpass
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.parse import urlencode
from urllib.parse import quote

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException


def scraping_cards_on_the_left(job_title="data", location="Barcelona, Catalonia, Spain"):
    options = webdriver.ChromeOptions()
    options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
    wd = webdriver.Chrome(options=options)
    
    url = 'https://www.linkedin.com/jobs/search/?currentJobId=3652284026&distance=25&geoId=107025191&keywords=data%20analyst&refresh=true'
    
    wd.get(url)
    no_of_jobs = int(wd.find_element(By.CSS_SELECTOR, "h1>span").get_attribute('innerText'))
    print("no_of_jobs: ", no_of_jobs)
    wd.find_element(By.CSS_SELECTOR, '.jobs-search__results-list')
    
    jobsitos = []
    els_jobs = []
    i = 1

    previous_page_content = ""  # Variable para almacenar el contenido de la página anterior

    while i <= int(no_of_jobs/25) + 1:
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        i = i + 1
        try:
            job_lists = wd.find_element(By.CSS_SELECTOR, '.jobs-search__results-list')
            listita = job_lists.get_attribute('outerHTML')
            soup = BeautifulSoup(listita, "html.parser")
            soup_2 = soup.find_all("ul", {"class": "jobs-search__results-list"})

            # 1. EXTRAER JOB A JOB
            for pagina in soup_2:
                jobsitos.append(pagina)

            # 2. EXTRAER LA INFO DE CADA JOB
            for job in jobsitos:
                job_list = job.find_all("li")

                for the_one_job in job_list:
                    url = the_one_job.find_all("a")[0].get("href")
                    title = the_one_job.find_all("span", {"class":"sr-only"})[0].text.strip()
                    location = the_one_job.find_all("span", {"class": "job-search-card__location" })[0].text.strip()
                    company = the_one_job.find_all("h4")[0].find("a").text.strip()
                    the_time = the_one_job.find_all("time")[0].text.strip()

                    dict_ = {
                        "title": title,
                        "company": company,
                        "location": location,
                        "time": the_time,
                        "url": url,
                    }

                    if dict_ not in els_jobs:
                        els_jobs.append(dict_)

            time.sleep(5)

            # Verificar si el botón "Ver más empleos" está presente y hacer clic en él
            show_more_button = wd.find_elements(By.CSS_SELECTOR, '.infinite-scroller__show-more-button')
            if show_more_button:
                show_more_button[0].click()
                time.sleep(5)
                
                # Verificar si el contenido de la página ha cambiado después de hacer clic
                current_page_content = wd.page_source

                if current_page_content == previous_page_content:
                    break  # Salir del bucle si no hay más trabajos disponibles

                previous_page_content = current_page_content

            else:
                break  # Salir del bucle si el botón no está presente

        except:
            pass
        time.sleep(5)

    os.system("say -v Monica don escreipin")
    return pd.DataFrame(els_jobs)

df_2 = scraping_cards_on_the_left()
df_2


The version of chrome cannot be detected. Trying with latest driver version


no_of_jobs:  2000


,title,company,location,time,url
0,Data Analyst,Exoticca,Barcelona,Hace 2 meses,https://es.linkedin.com/jobs/view/data-analyst...
1,Junior Analytics Engineer,Exoticca,Barcelona,Hace 3 semanas,https://es.linkedin.com/jobs/view/junior-analy...
2,Data Analyst,HP,Sant Cugat del Vallès,Hace 7 horas,https://es.linkedin.com/jobs/view/data-analyst...
3,Analytics Engineer,Exoticca,Barcelona,Hace 3 meses,https://es.linkedin.com/jobs/view/analytics-en...
4,Data Scientist (Andorra),Gauss & Neumann,Barcelona,Hace 1 mes,https://es.linkedin.com/jobs/view/data-scienti...
...,...,...,...,...,...
382,Junior Engineer,Submer,Rubí,Hace 1 semana,https://es.linkedin.com/jobs/view/junior-engin...
383,Big Data Software Engineer,Bridgestone Mobility Solutions,Barcelona,Hace 3 semanas,https://es.linkedin.com/jobs/view/big-data-sof...
384,Data Engineer Senior,KPMG España,Hospitalet de Llobregat,Hace 1 semana,https://es.linkedin.com/jobs/view/data-enginee...
385,STAGER DATA SCIENTIST,MANGO,Barcelona,Hace 3 semanas,https://es.linkedin.com/jobs/view/stager-data-...


In [32]:
df_2.to_csv('ultimo_funcion_1.csv', index=False)

In [39]:
df_last = pd.read_csv('ultimo_funcion_1.csv')


In [49]:
df_primerexito = pd.read_csv('primer_exito_bro_to_wapo.csv')

In [46]:
df_medio = pd.read_csv('all_data_analysts_jobs175.csv')

In [42]:
df_primero = pd.read_csv('all_data_analysts_jobs.csv')

In [ ]:
df.to_csv('all_data_analysts_jobs175.csv', index=False)

In [50]:
df_combined = pd.concat([df_last, df_primero, df_medio, df_primerexito]).drop_duplicates()


In [52]:
df_combined.to_csv('first_function_datajobs.csv', index=False)

,title,company,location,time,url,Unnamed: 0
0,Data Analyst,Exoticca,Barcelona,Hace 2 meses,https://es.linkedin.com/jobs/view/data-analyst...,NaN
1,Junior Analytics Engineer,Exoticca,Barcelona,Hace 3 semanas,https://es.linkedin.com/jobs/view/junior-analy...,NaN
2,Data Analyst,HP,Sant Cugat del Vallès,Hace 7 horas,https://es.linkedin.com/jobs/view/data-analyst...,NaN
3,Analytics Engineer,Exoticca,Barcelona,Hace 3 meses,https://es.linkedin.com/jobs/view/analytics-en...,NaN
4,Data Scientist (Andorra),Gauss & Neumann,Barcelona,Hace 1 mes,https://es.linkedin.com/jobs/view/data-scienti...,NaN
...,...,...,...,...,...,...
245,Data Engineer,Exoticca,Barcelona,Hace 3 semanas,https://es.linkedin.com/jobs/view/data-enginee...,245.0
246,European Data Analyst,Diversey,Viladecans,Hace 1 mes,https://es.linkedin.com/jobs/view/european-dat...,246.0
247,Senior Data Analyst,Zurich Insurance,Barcelona,Hace 2 semanas,https://es.linkedin.com/jobs/view/senior-data-...,247.0
248,Power BI Consultant (I&D),Capgemini,Barcelona,Hace 1 semana,https://es.linkedin.com/jobs/view/power-bi-con...,248.0


In [62]:
df_combined_2 = df_combined.reset_index(drop=True)

In [74]:
df_combined_2.to_csv('csv_urls.csv', index=False)


In [75]:
df_combined_2.url[0]

'https://es.linkedin.com/jobs/view/data-analyst-at-exoticca-3622172076?refId=jWf2Fweu6LwaRUAHa5CNVQ%3D%3D&trackingId=vzc7hXRuY8DFO0xPl1w3VA%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card'

# Scrapear detalles del job

options = webdriver.ChromeOptions()
options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
wd = webdriver.Chrome(options=options)
url_job = "https://es.linkedin.com/jobs/view/data-analyst-at-coniq-3652827423?refId=LmHLqc2XCC2YJGeJ%2BOJlfw%3D%3D&trackingId=CtHhJ%2Fe3ZQTxs8KZpryElw%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card"
wd.get(url_job)

#Description’: jd
#Seniority
#Type’: emp_type,
#Function’: job_func,

def logging_in (url):
    
    # 1. Initializing driver
    options = webdriver.ChromeOptions()
    options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
    wd = webdriver.Chrome(options=options)
    wd.get(url)
    
    # 2. Try to log-in
    
    try:
        time.sleep(5)
        driver.find_element(By.XPATH,'/html/body/header/nav/div/a[2]').click()
        mail = 'cocazapata.21@gmail.com'
        pw = 'fcbarcelona21'
        time.sleep(3)
        print("Logging in")

        username = wait.until(EC.visibility_of_element_located((By.ID, 'username'))).send_keys(mail)
        password = wait.until(EC.visibility_of_element_located((By.ID, 'password'))).send_keys(pw)
        password.send_keys(Keys.RETURN)
        time.sleep(2)
        return wd.page_source
        
    # 3. Otherwise: try to just get the content
        
    except:
        print("Not logging in")
        time.sleep(2)
        return wd.page_source

import requests
from bs4 import BeautifulSoup
import time

def info_un_job(url):
    
    response = logging_in (url)
    soup = BeautifulSoup(response, 'html.parser')
    descripcion = soup.find('div', {"class":'description__text description__text--rich'}).get_text(strip=True)
    nivel_antiguedad = soup.find('span', {"class":"description__job-criteria-text description__job-criteria-text--criteria"}).get_text(strip=True)
    sectores = soup.find('span', {"class":"description__job-criteria-text description__job-criteria-text--criteria"})
    
    dict_ = {
        "description": descripcion,
        "nivel_antiguedad": nivel_antiguedad,
        "sectores": sectores
    }
    
    return dict_
    

url = 'https://es.linkedin.com/jobs/view/data-analyst-at-coniq-3652827423?refId=LmHLqc2XCC2YJGeJ%2BOJlfw%3D%3D&trackingId=CtHhJ%2Fe3ZQTxs8KZpryElw%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card'

resultado = info_un_job(url)
resultado


In [69]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from datetime import datetime

def logging_in(url):
    options = webdriver.ChromeOptions()
    options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
    wd = webdriver.Chrome(options=options)
    wd.get(url)

    try:
        time.sleep(5)
        wd.find_element(By.XPATH, '/html/body/header/nav/div/a[2]').click()
        mail = 'cocazapata.21@gmail.com'
        pw = 'fcbarcelona21'
        time.sleep(3)
        print("Logging in")

        username = wd.find_element(By.ID, 'username')
        username.send_keys(mail)
        password = wd.find_element(By.ID, 'password')
        password.send_keys(pw)
        password.send_keys(Keys.RETURN)
        time.sleep(2)
        return wd
    except:
        print("Not logging in")
        time.sleep(2)
        return wd


def info_un_job(url):
    wd = logging_in(url)
    
    response = wd.page_source
    soup = BeautifulSoup(response, 'html.parser')
    
    company = soup.find('span', {"class":"jobs-unified-top-card__company-name"}).get_text(strip=True)
    position = soup.find('h1', {"class":"t-24 t-bold jobs-unified-top-card__job-title"}).get_text(strip=True)
    location = soup.find('span', {"class":"jobs-unified-top-card__bullet"}).get_text(strip=True)
    workplace = soup.find('span', {"class":"jobs-unified-top-card__workplace-type"}).get_text(strip=True)
    type_and_level = soup.find("li", {"class": "jobs-unified-top-card__job-insight"}).find("span").text.strip()
    description = soup.find('div', {"class": 'jobs-box__html-content jobs-description-content__text t-14 t-normal jobs-description-content__text--stretch'}).get_text(strip=True)
    date = soup.find('span', {"class":"jobs-unified-top-card__posted-date"}).get_text(strip=True)
    today = datetime.today().strftime('%d-%m-%Y')
    link_text = soup.find('a', class_='app-aware-link').text.strip()
    #workers = soup.find("li", {"class": "jobs-unified-top-card__job-insight"}).find("span").text.strip()
    #num_alumnis = link_text.split()[0]
 
     
    # Handling missing elements
    applicants = soup.find('span', class_='jobs-unified-top-card__applicant-count')
    applicants = applicants.text.strip() if applicants else None
    
    dict_2 = {
        "company" : company,
        "position": position,
        "location": location,
        "workplace" : workplace,
        "description": description,
        "type_and_level": type_and_level,
        "date": date,
        "today": today,
        #"num_alumnis": num_alumnis,
        "applicants": applicants,
        #"workers": workers,
        "url": url,
    }
    
    os.system("say -v Monica don escreipin")

    return dict_2

url = 'https://es.linkedin.com/jobs/view/data-analyst-at-coniq-3652827423?refId=LmHLqc2XCC2YJGeJ%2BOJlfw%3D%3D&trackingId=CtHhJ%2Fe3ZQTxs8KZpryElw%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card'
resultado = info_un_job(url)
df_test = pd.DataFrame([resultado])
df_test

df_test

The version of chrome cannot be detected. Trying with latest driver version


Logging in


,company,position,location,workplace,description,type_and_level,date,today,applicants,url
0,Coniq,Data Analyst,"Barcelona, Catalonia, Spain",On-site,"About the jobConiq is a dynamic, high growth U...",Full-time · Associate,1 month ago,11-07-2023,88 applicants,https://es.linkedin.com/jobs/view/data-analyst...


In [71]:
resultado

{'company': 'Coniq',
 'position': 'Data Analyst',
 'location': 'Barcelona, Catalonia, Spain',
 'workplace': 'On-site',
 'description': 'About the jobConiq is a dynamic, high growth UK-based SaaS company that provides total customer engagement and loyalty services to many of the world’s leading property developers, shopping centers and retailers. Coniq powers well over £1 billion of sales annually for its clients, with more than 20 million consumers shopping at 1,800 brands in 24 countries worldwide, and has offices in Europe, the US and the Middle East.Coniq is looking for a Data Analst to join our brilliant team in Barcelona. We have a robust data infrastructure and we need someone to help us develop our industry-leading product offerings and capabilities to the next level. If you’re looking for a position which will offer variety, complexity, responsibility and challenges, this could be the role for you and we’d love to start a conversation!ResponsibilitiesDevelop the roadmap for our

In [ ]:
import pandas as pd
import concurrent.futures

# Cargar el DataFrame con las URLs
csv_urls = pd.read_csv('csv_urls.csv')

# Función para procesar una URL y extraer la información de la vacante
def procesar_url(url):
    resultado = info_un_job(url)
    return resultado

# Especificar el número de hilos a utilizar
num_hilos = 8

# Crear un ThreadPoolExecutor con el número de hilos especificado
executor = concurrent.futures.ThreadPoolExecutor(max_workers=num_hilos)

# Crear una lista para almacenar los resultados
results = []

# Iterar sobre la columna "url" y enviar cada URL al ThreadPoolExecutor para procesarla en paralelo
for url in csv_urls['url']:
    future = executor.submit(procesar_url, url)
    results.append(future)

# Obtener los resultados a medida que estén listos
df_results = pd.DataFrame([future.result() for future in concurrent.futures.as_completed(results)])

# Mostrar el DataFrame resultante
print(df_results)








The version of chrome cannot be detected. Trying with latest driver version
The version of chrome cannot be detected. Trying with latest driver version
The version of chrome cannot be detected. Trying with latest driver version
The version of chrome cannot be detected. Trying with latest driver version
The version of chrome cannot be detected. Trying with latest driver version
The version of chrome cannot be detected. Trying with latest driver version
The version of chrome cannot be detected. Trying with latest driver version
The version of chrome cannot be detected. Trying with latest driver version


Logging in
Logging in
Logging in
Logging in
Logging in
Logging in
Logging in


In [ ]:
import pandas as pd
import time

# Assuming "all_data_analysts_jobs" is the dataframe containing the URLs
all_data_analysts_jobs =pd.read_csv('all_data_analysts_jobs.csv') # Replace ... with your actual dataframe creation code

# Create an empty dataframe to store the extracted information
extracted_data = pd.DataFrame(columns=["company", "position", "location", "workplace", "description", "type_and_level", "date"])

# Iterate over each URL in the dataframe
for index, row in all_data_analysts_jobs.iterrows():
    url = row['url']
    wd = logging_in(url)
    resultado = info_un_job(url, wd)
    
    # Append the extracted information to the dataframe
    extracted_data = extracted_data.append(resultado, ignore_index=True)
    
    # Add a delay of 5 seconds between each iteration
    time.sleep(5)

# Save the extracted data to a new dataframe or a file
extracted_data.to_csv("extracted_data.csv", index=False)  # Example: Save as CSV file
# extracted_data.to_excel("extracted_data.xlsx", index=False)  # Example: Save as Excel file


jobsasos = []

for item in range(len(jobs)):

    # Haciendo clic en el trabajo para ver los detalles del trabajo
    job_click_path = f'/html/body/main/div/section[2]/ul/li[{item+1}]/img'
    job_click = jobs[item].find_element_by_xpath(job_click_path).click()
    time.sleep(5)

    jd_path = '/html/body/main/section/div[2]/section[2]/div'
    jd = wd.find_element_by_xpath(jd_path).get_attribute('innerText')
    jd.append(jd)

    seniority_path = '/html/body/main/section/div[2]/section[2]/ul/li[1]/span'
    seniority = wd.find_element_by_xpath(seniority_path).get_attribute('innerText')
    seniority.append(seniority)

    emp_type_path = '/html/body/main/section/div[2]/section[2]/ul/li[2]/span'
    emp_type = wd.find_element_by_xpath(emp_type_path).get_attribute('innerText')
    emp_type.append(emp_type)

    job_func_path = '/html/body/main/section/div[2]/section[2]/ul/li[3]/span'
    job_func_elements = wd.find_elements_by_xpath(job_func_path)
    for element in job_func_elements:
        job_func.append(element.get_attribute('innerText'))
    job_func_final = ', '.join(job_func)
    job_func.append(job_func_final)

    industries_path = '/html/body/main/section/div[2]/section[2]/ul/li[4]/span'
    industries_elements = wd.find_elements_by_xpath(industries_path)
    for element in industries_elements:
        industries.append(element.get_attribute('innerText'))
    industries_final = ', '.join(industries)
    #industries.append(industries_final)

   
    


In [ ]:
import pandas as pd

job_data = pd.DataFrame({
    'ID': job_id,
    'Date': date,
    'Company': company_name,
    'Title': job_title,
    'Location': location,
    'Description': jd,
    'Level': seniority,
    'Type': emp_type,
    'Function': job_func,
    'Industry': industries,
    'Link': job_link
})

# Limpiar la columna "Description"
job_data['Description'] = job_data['Description'].str.replace('\n', ' ')

job_data.to_excel('LinkedIn Job Data_Data Scientist.xlsx', index=False)


In [ ]:
job_data = pd.DataFrame({‘ID’: job_id,
‘Date’: date,
‘Company’: company_name,
‘Title’: job_title,
‘Location’: location,
'Description’: jd,
‘Level’: seniority,
‘Type’: emp_type,
‘Function’: job_func,
‘Industry’: industries,
‘Link’: job_link
})
# cleaning description column
job_data[‘Description’] = job_data[‘Description’].str.replace(‘\n’,’ ‘)
job_data.to_excel('LinkedIn Job Data_Data Scientist.xlsx', index = False)

```python
# Data management
import pandas as pd
import string

# Databases
import sqlalchemy as alch
from getpass import getpass
from pymongo import MongoClient

from IPython.display import Image


# Languages
import re

import spacy
#import es_core_news_sm

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

from wordcloud import WordCloud
from langdetect import detect
from textblob import TextBlob

from nltk import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

text = ["I love writing code in Python. I love Python code",
        "I hate writing code in Java. I hate Java code"]


df = pd.DataFrame({'review': ['review1', 'review2'], 'text':text})


cv = CountVectorizer(stop_words='english')
cv_matrix = cv.fit_transform(df['text'])
df_dtm = pd.DataFrame(cv_matrix.toarray(),
                      index=df['review'].values,
                      columns=cv.get_feature_names())
df_dtm
```